In [1]:
import pandas as pd
import sqlite3
from datetime import datetime

In [128]:
def setup_database(db):
    """
    Set up the initial database schema.
    """
    cursor = db.cursor()
    
    # create a household table to store household information
    cursor.execute("""--sql
        CREATE TABLE IF NOT EXISTS households (
            household_id INTEGER PRIMARY KEY AUTOINCREMENT,
            household_name TEXT UNIQUE NOT NULL
        );
    """)
    
    # Create members table
    cursor.execute("""--sql
        CREATE TABLE IF NOT EXISTS members (
            member_id INTEGER PRIMARY KEY AUTOINCREMENT,
            member_name TEXT NOT NULL,
            chore_order INTEGER,
            household_id INTEGER NOT NULL,
            FOREIGN KEY (household_id) REFERENCES households(household_id),
            UNIQUE(member_name, household_id)
        );
    """)
    
    # Create chores table
    cursor.execute("""--sql
        CREATE TABLE IF NOT EXISTS chores (
            chore_id INTEGER PRIMARY KEY AUTOINCREMENT,
            chore_name TEXT NOT NULL,
            last_done_date DATE,
            frequency INTEGER,
            previous_member_id INTEGER,
            due_member_id INTEGER,
            next_member_id INTEGER,
            household_id INTEGER,
            FOREIGN KEY (household_id) REFERENCES households(household_id),
            FOREIGN KEY (previous_member_id) REFERENCES members(member_id),
            FOREIGN KEY (due_member_id) REFERENCES members(member_id),
            FOREIGN KEY (next_member_id) REFERENCES members(member_id),
            UNIQUE(chore_name, household_id)
        );
    """)
    
    # Create chores_stats table
    cursor.execute("""--sql
        CREATE TABLE IF NOT EXISTS chores_stats (
            member_id INTEGER,
            chore_id INTEGER,
            done_count INTEGER DEFAULT 0,
            avg_delay FLOAT DEFAULT 0,
            last_done_date DATE,
            PRIMARY KEY (member_id, chore_id)
        );
    """)
    
    # Create chores_log table
    cursor.execute("""--sql
        CREATE TABLE IF NOT EXISTS chores_log (
            log_id INTEGER PRIMARY KEY AUTOINCREMENT,
            chore_id INTEGER,
            completing_member_id INTEGER,
            due_member_id INTEGER,
            completion_date DATE,
            due_date DATE,
            delay INTEGER
        );
    """)
    
    db.commit()


def add_household(db, household_name) -> int:
    """
    Add a household to the database if it doesn't already exist.
    """
    cursor = db.cursor()
    
    # Insert household if it doesn't already exist
    cursor.execute("""--sql
        INSERT INTO households (household_name)
        VALUES (?);
    """, (household_name,))
    
    # get the household_id of the newly added household
    cursor.execute("""--sql
        SELECT household_id FROM households WHERE household_name = ?;
    """, (household_name,))
    household_id = cursor.fetchone()[0]
    
    # print the households table
    print("households:")
    display(pd.read_sql("SELECT * FROM households;", db))

    db.commit()

    return household_id


def add_member(db, household_name, member_name) -> int:
    """
    Add a member to the database if they don't already exist.
    """
    cursor = db.cursor()
    
    # Get household_id from household_name
    cursor.execute("""--sql
        SELECT household_id FROM households WHERE household_name = ?;
    """, (household_name,))
    household_id = cursor.fetchone()[0]
    
    # Insert member if they don't already exist
    cursor.execute("""--sql
        INSERT INTO members (member_name, household_id)
        VALUES (?, ?);
    """, (member_name, household_id))
    
    # print the members table
    print("members:")
    display(pd.read_sql("SELECT * FROM members;", db))
    
    # get the member_id of the newly added member
    cursor.execute("""--sql
        SELECT member_id FROM members WHERE member_name = ? AND household_id = ?;
    """, (member_name, household_id))
    member_id = cursor.fetchone()[0]

    db.commit()

    return member_id


def remove_member(db, household_name, member_name):
    """
    Remove a member from the database.
    """
    cursor = db.cursor()
    
    # Get household_id from household_name
    cursor.execute("""--sql
        SELECT household_id FROM households WHERE household_name = ?;
    """, (household_name,))
    household_id = cursor.fetchone()
    
    # Get member_id from member_name and household_id
    cursor.execute("""--sql
        SELECT member_id FROM members WHERE member_name = ? AND household_id = ?;
    """, (member_name, household_id))
    member_id = cursor.fetchone()
    
    # Delete the member from the members table
    cursor.execute("""--sql
        DELETE FROM members WHERE member_name = ? AND household_id = ?;
    """, (member_name, household_id))

    # Also remove any stats related to this member
    cursor.execute("""--sql
        DELETE FROM chores_stats WHERE member_id = ?;
    """, (member_id,))
    
    # Update the chore order for remaining members in the members table
    cursor.execute("""--sql
        UPDATE members
        SET chore_order = chore_order - 1
        WHERE chore_order > (SELECT chore_order FROM members WHERE member_name = ?);
    """, (member_name,))
    
    # TODO: Update the chores table to remove any references to this member
    
    db.commit()
    

def set_chore_order(db, household_name, member_names):
    """Set the order of members for chores (based on order of the list)."""
    cursor = db.cursor()

    # get household_id from household_name
    cursor.execute("""--sql
        SELECT household_id FROM households WHERE household_name = ?;
    """, (household_name,))
    household_id = cursor.fetchone()[0]

    # update each member's chore_order according to its position in the list
    for i, member in enumerate(member_names):
        cursor.execute("""--sql
            UPDATE members
            SET chore_order = ?
            WHERE member_name = ? AND household_id = ?;
        """, (i, member, household_id))

    db.commit()
    
    
def add_chores(db, household_name, chore_name, starting_date, frequency) -> int:
    """
    Add a chore to the chores table.
    """
    cursor = db.cursor()
    
    # Get household_id from household_name
    cursor.execute("""--sql
        SELECT household_id FROM households WHERE household_name = ?;
    """, (household_name,))
    household_id = cursor.fetchone()[0]
    
    # Insert the chore into the chores table
    cursor.execute("""--sql
        INSERT INTO chores (chore_name, last_done_date, frequency, household_id)
        VALUES (?, ?, ?, ?);
    """, (chore_name, starting_date, frequency, household_id))
    
    # get the chore_id of the newly added chore
    cursor.execute("""--sql
        SELECT chore_id FROM chores WHERE chore_name = ? AND household_id = ?;
    """, (chore_name, household_id))
    chore_id = cursor.fetchone()[0]
    
    # Insert initial stats for each member
    cursor.execute("""--sql
        INSERT OR IGNORE INTO chores_stats (member_id, chore_id, done_count, avg_delay, last_done_date)
        SELECT member_id, ?, 0, 0, DATE('now') FROM members WHERE household_id = ?;
    """, (chore_id, household_id))
    
    # get the due member and next member for this chore
    due_member_id = get_due_member(db, household_name, chore_name, get_next=False)
    next_member_id = get_due_member(db, household_name, chore_name, get_next=True)
    
    # insert the due member and next member into the chores table
    cursor.execute("""--sql
        UPDATE chores
        SET due_member_id = ?, next_member_id = ?
        WHERE chore_id = ?;
    """, (due_member_id, next_member_id, chore_id))
    
    # print the chores table
    print("chores:")
    display(pd.read_sql("SELECT * FROM chores;", db))
    
    db.commit()
    
    return chore_id


def get_due_member(db, household_name, chore_name, get_next=False) -> int:
    """
    Get the due member for a chore based on:
    1. Whoever has done the chore the least
    2. If tied, whoever comes first in the member order
    
    after_due: If True, return the member after the next one in line.
    """
    
    cursor = db.cursor()
    
    # Get household_id from household_name
    cursor.execute("""--sql
        SELECT household_id FROM households WHERE household_name = ?;
    """, (household_name,))
    household_id = cursor.fetchone()[0]
    
    # Fetch the chore_id for the given chore_name
    cursor.execute("""--sql
        SELECT chore_id FROM chores WHERE chore_name = ?;
    """, (chore_name,))
    chore_id = cursor.fetchone()[0]
    
    # Get the member with the lowest done_count for the chore
    cursor.execute("""--sql
        SELECT l.member_id
        FROM chores_stats l
        INNER JOIN members r ON l.member_id = r.member_id
        WHERE l.chore_id = ? AND r.household_id = ?
        ORDER BY l.done_count, r.chore_order
        LIMIT 1;
    """, (chore_id, household_id))
    due_member_id = cursor.fetchone()[0]
    
    if get_next:
        # temporarily update the done_count for the new due member, to identify the next member
        cursor.execute("""--sql
            UPDATE chores_stats
            SET done_count = done_count + 1
            WHERE member_id = ? AND chore_id = ?;
        """, (due_member_id, chore_id))

        # Get the next member after the due one
        next_member_id = get_due_member(db, household_name, chore_name, get_next=False)
        
        # Restore the done_count for the new due member
        cursor.execute("""--sql
            UPDATE chores_stats
            SET done_count = done_count - 1
            WHERE member_id = ? AND chore_id = ?;
        """, (due_member_id, chore_id))

        return next_member_id
    else:
        return due_member_id


def update_chore_completion(db, household_name, completing_member, chore_name, completion_date):
    """
    Update the database when a chore is completed
    """
    cursor = db.cursor()
    
    # Get household_id from household_name
    cursor.execute("""--sql
        SELECT household_id 
        FROM households 
        WHERE household_name = ?;
    """, (household_name,))
    household_id = cursor.fetchone()[0]
    
    # get the member_id for the completing member
    cursor.execute("""--sql
        SELECT member_id 
        FROM members 
        WHERE member_name = ? AND household_id = ?;
    """, (completing_member, household_id))
    completing_member_id = cursor.fetchone()[0]

    # determine the member who is supposed to complete the chore
    cursor.execute("""--sql
        SELECT due_member_id, chore_id
        FROM chores 
        WHERE chore_name = ? AND household_id = ?;
    """, (chore_name, household_id))
    due_member_id, chore_id = cursor.fetchone()

    # Calculate delay and due date
    cursor.execute("""--sql
        SELECT 
            (JulianDay(?) - (JulianDay(last_done_date) + frequency)) AS delay,
            date(JulianDay(last_done_date) + frequency) AS due_date
        FROM chores 
        WHERE chore_id = ?;
    """, (completion_date, chore_id))
    delay, due_date = cursor.fetchone()
    
    # Log the completion
    cursor.execute("""--sql
        INSERT INTO chores_log (chore_id, completing_member_id, due_member_id, completion_date, due_date, delay)
        VALUES (?, ?, ?, ?, ?, ?);
    """, (chore_id, completing_member_id, due_member_id, completion_date, due_date, delay))
    
    # Update donecount for the completing member
    cursor.execute("""--sql
        UPDATE chores_stats
        SET done_count = done_count + 1, last_done_date = ?
        WHERE member_id = ? AND chore_id = ?;
    """, (completion_date, completing_member_id, chore_id))

    if due_member_id == completing_member_id:
        # the done count has already been incremented, so we use done_count - 1 to calculate the previous sum
        # of the average
        cursor.execute("""--sql
            UPDATE chores_stats
            SET avg_delay = (avg_delay * (done_count - 1) + ?) / done_count
            WHERE member_id = ? AND chore_id = ?;
        """, (delay, completing_member_id, chore_id))
    else:
        # If someone else completed the chore, the delay is added to the member who was supposed to do it.
        # However, the done count was not incremented for that member, so we need to slightly adjust the 
        # formula.
        cursor.execute("""--sql
            UPDATE chores_stats
            SET avg_delay = (avg_delay * done_count + ?) / (done_count + 1)
            WHERE member_id = ? AND chore_id = ?;
        """, (delay, completing_member_id, chore_id))

    # Get the next member
    due_member_id = get_due_member(db, household_name, chore_name, get_next=False)
    next_member_id = get_due_member(db, household_name, chore_name, get_next=True)

    # Update the main chores table
    cursor.execute("""--sql
        UPDATE chores
        SET last_done_date = ?, 
            previous_member_id = ?, 
            due_member_id = ?, 
            next_member_id = ?
        WHERE chore_id = ?;
    """, (completion_date, completing_member_id, due_member_id, next_member_id, chore_id))

    db.commit()

In [129]:
import os
try:
    os.remove('aemtlis.db')
    os.remove('aemtlis.db-journal')
except:
    pass
db = sqlite3.connect('aemtlis.db')
setup_database(db)

In [130]:
# create a household
household_name = 'WG'
household_id = add_household(db, household_name)

# add members
member_id_ricco = add_member(db, household_name, 'Ricco')
member_id_lorin = add_member(db, household_name, 'Lorin')
member_id_niccolo = add_member(db, household_name, 'Niccolo')

# set the order of members
set_chore_order(db, household_name, ['Ricco', 'Lorin', 'Niccolo'])

# add chore
chore_1_name = 'Wohnung putzen'
chore_1_id = add_chores(db, household_name, chore_1_name, '2025-08-01', 7)

households:


,household_id,household_name
0,1,WG


members:


,member_id,member_name,chore_order,household_id
0,1,Ricco,None,1


members:


,member_id,member_name,chore_order,household_id
0,1,Ricco,None,1
1,2,Lorin,None,1


members:


,member_id,member_name,chore_order,household_id
0,1,Ricco,None,1
1,2,Lorin,None,1
2,3,Niccolo,None,1


chores:


,chore_id,chore_name,last_done_date,frequency,previous_member_id,due_member_id,next_member_id,household_id
0,1,Wohnung putzen,2025-08-01,7,None,1,2,1


In [131]:
display(pd.read_sql("SELECT * FROM members", db))
display(pd.read_sql("SELECT * FROM chores_stats", db))
display(pd.read_sql("SELECT * FROM chores", db))
display(pd.read_sql("SELECT * FROM chores_log", db))

,member_id,member_name,chore_order,household_id
0,1,Ricco,0,1
1,2,Lorin,1,1
2,3,Niccolo,2,1


,member_id,chore_id,done_count,avg_delay,last_done_date
0,2,1,0,0.0,2025-09-03
1,3,1,0,0.0,2025-09-03
2,1,1,0,0.0,2025-09-03


,chore_id,chore_name,last_done_date,frequency,previous_member_id,due_member_id,next_member_id,household_id
0,1,Wohnung putzen,2025-08-01,7,None,1,2,1


,log_id,chore_id,completing_member_id,due_member_id,completion_date,due_date,delay


In [132]:
print(f'Should: {get_due_member(db, household_name,chore_1_name)}, done by Ricco')
update_chore_completion(db, household_name, 'Ricco', chore_1_name, '2025-08-08')
display(pd.read_sql("SELECT * FROM chores_stats", db))
print(f'Should: {get_due_member(db, household_name, chore_1_name)}, done by Lorin')
update_chore_completion(db, household_name, 'Lorin', chore_1_name, '2025-08-16')
display(pd.read_sql("SELECT * FROM chores_stats", db))
print(f'Should: {get_due_member(db, household_name, chore_1_name)}, done by Niccolo')
update_chore_completion(db, household_name, 'Niccolo', chore_1_name, '2025-08-24')
display(pd.read_sql("SELECT * FROM chores_stats", db))
print(f'Should: {get_due_member(db, household_name, chore_1_name)}, done by Ricco')
update_chore_completion(db, household_name, 'Ricco', chore_1_name, '2025-08-30')
display(pd.read_sql("SELECT * FROM chores_stats", db))
print(f'Should: {get_due_member(db, household_name, chore_1_name)}, done by Ricco')
update_chore_completion(db, household_name, 'Ricco', chore_1_name, '2025-09-06')
display(pd.read_sql("SELECT * FROM chores_stats", db))
print(f'Should: {get_due_member(db, household_name, chore_1_name)}, done by Lorin')
update_chore_completion(db, household_name, 'Lorin', chore_1_name, '2025-09-16')
display(pd.read_sql("SELECT * FROM chores_stats", db))
print(f'Should: {get_due_member(db, household_name, chore_1_name)}, done by Niccolo')
update_chore_completion(db, household_name, 'Niccolo', chore_1_name, '2025-09-24')
display(pd.read_sql("SELECT * FROM chores_stats", db))
display(pd.read_sql("SELECT * FROM chores_log", db))


Should: 1, done by Ricco


,member_id,chore_id,done_count,avg_delay,last_done_date
0,2,1,0,0.0,2025-09-03
1,3,1,0,0.0,2025-09-03
2,1,1,1,0.0,2025-08-08


Should: 2, done by Lorin


,member_id,chore_id,done_count,avg_delay,last_done_date
0,2,1,1,1.0,2025-08-16
1,3,1,0,0.0,2025-09-03
2,1,1,1,0.0,2025-08-08


Should: 3, done by Niccolo


...

Should: 2, done by Lorin


,member_id,chore_id,done_count,avg_delay,last_done_date
0,2,1,2,2.000,2025-09-16
1,3,1,1,1.000,2025-08-24
2,1,1,3,-0.375,2025-09-06


Should: 3, done by Niccolo


,member_id,chore_id,done_count,avg_delay,last_done_date
0,2,1,2,2.000,2025-09-16
1,3,1,2,1.000,2025-09-24
2,1,1,3,-0.375,2025-09-06


,log_id,chore_id,completing_member_id,due_member_id,completion_date,due_date,delay
0,1,1,1,1,2025-08-08,2025-08-08,0
1,2,1,2,2,2025-08-16,2025-08-15,1
2,3,1,3,3,2025-08-24,2025-08-23,1
3,4,1,1,1,2025-08-30,2025-08-31,-1
4,5,1,1,2,2025-09-06,2025-09-06,0
5,6,1,2,2,2025-09-16,2025-09-13,3
6,7,1,3,3,2025-09-24,2025-09-23,1
